In [28]:
import os
import os.path as osp

import torch
import torch.nn as nn
import torch.optim as optim                                                      

import numpy as np
import pandas as pd


from dataset import load_data
from config import config as conf


class Flatten(nn.Module):
    def forward(self, x):
        return x.reshape(x.shape[0], -1)
    def get_embedding(self, x):
        return self.forward(x)


class SEConv(nn.Module):
    """Use Convolution instead of FullyConnection in SE"""

    def __init__(self, channels, reduction):
        super().__init__()
        self.net = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Conv2d(channels, channels // reduction, kernel_size=1, bias=False),
            nn.ReLU(inplace=True),
            nn.Conv2d(channels // reduction, channels, kernel_size=1, bias=False),
            nn.Sigmoid(),
        )

    def forward(self, x):
        return self.net(x) * x

    def get_embedding(self, x):
        return self.forward(x)


class SE(nn.Module):

    def __init__(self, channels, reduction):
        super().__init__()
        self.net = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Linear(channels, channels // reduction),
            nn.ReLU(inplace=True),
            nn.Linear(channels // reduction, channels),
            nn.Sigmoid(),
        )

    def forward(self, x):
        return self.net(x) * x


class IRSE(nn.Module):

    def __init__(self, channels, depth, stride):
        super().__init__()
        if channels == depth:
            self.shortcut = nn.MaxPool2d(kernel_size=1, stride=stride)
        else:
            self.shortcut = nn.Sequential(
                nn.Conv2d(channels, depth, (1, 1), stride, bias=False), 
                nn.BatchNorm2d(depth),
            )
        self.residual = nn.Sequential(
            nn.BatchNorm2d(channels),
            nn.Conv2d(channels, depth, (3, 3), 1, 1, bias=False),
            nn.PReLU(depth),
            nn.Conv2d(depth, depth, (3, 3), stride, 1, bias=False),
            nn.BatchNorm2d(depth),
            SEConv(depth, 16),
        )

    def forward(self, x):
        return self.shortcut(x) + self.residual(x)

class ResIRSE(nn.Module):
    """Resnet50-IRSE backbone"""

    def __init__(self, embedding_size, drop_ratio):
        super().__init__()
        self.input_layer = nn.Sequential(
            nn.Conv2d(1, 64, (3, 3), 1, 1, bias=False),
            nn.BatchNorm2d(64),
            nn.PReLU(64),
        )
        self.output_layer = nn.Sequential(
            nn.BatchNorm2d(512),
            nn.Dropout(drop_ratio),
            Flatten(),
            nn.Linear(512 * 8 * 8, embedding_size),
            nn.BatchNorm1d(embedding_size),
        )

        # ["channels", "depth", "stride"],
        self.res50_arch = [
            [64, 64, 2], [64, 64, 1], [64, 64, 1],
            [64, 128, 2], [128, 128, 1], [128, 128, 1], [128, 128, 1],
            [128, 256, 2], [256, 256, 1], [256, 256, 1], [256, 256, 1], [256, 256, 1],
            [256, 256, 1], [256, 256, 1], [256, 256, 1], [256, 256, 1], [256, 256, 1],
            [256, 256, 1], [256, 256, 1], [256, 256, 1], [256, 256, 1],
            [256, 512, 2], [512, 512, 1], [512, 512, 1],
        ]

        self.body = nn.Sequential(*[ IRSE(a,b,c) for (a,b,c) in self.res50_arch ])
    def forward(self, x):
        x = self.input_layer(x)
        x = self.body(x)
        x = self.output_layer(x) 
        return x
    def get_embedding(self, x):
        return self.forward(x)

# device = conf.device
# embedding_size = conf.embedding_size
# checkpoints = conf.checkpoints
device = 'cuda' if torch.cuda.is_available() else 'cpu'
weights_path = "checkpoints/arcfacemodel.pth"
net = ResIRSE(512, 0.5).to(device)
net= nn.DataParallel(net)
# path = r'checkpoints/arcfacemodel.pth'
net.load_state_dict(torch.load(weights_path, map_location=device))
print(net)


DataParallel(
  (module): ResIRSE(
    (input_layer): Sequential(
      (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): PReLU(num_parameters=64)
    )
    (output_layer): Sequential(
      (0): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (1): Dropout(p=0.5, inplace=False)
      (2): Flatten()
      (3): Linear(in_features=32768, out_features=512, bias=True)
      (4): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (body): Sequential(
      (0): IRSE(
        (shortcut): MaxPool2d(kernel_size=1, stride=2, padding=0, dilation=1, ceil_mode=False)
        (residual): Sequential(
          (0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
       